In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2023-03-01'
final_date = '2023-03-31'
query_error_year = 2023
query_error_month = 3

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2023\03. MARZO\DATA\ERROR MARZO 2023.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(118914, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21464\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
112083,112083,2887ea44-8d27-45f3-9654-dadf290e75be,Freddo Condesa,D3H2CYW8R,2023-03-01,"1,176.71",ERROR,DEVOLUCION(02): Cuenta bloqueada 021180...,2023-03-01,3,2023,NEW,ERROR_USUAL,2887ea44-8d27-45f3-9654-dadf290e75be,2023-03-01,1,Actual
112047,112047,0c7b3b18-52e0-4678-8b81-a5638037781a,Recargas y zapato s,MMLMYGB2D,2023-03-01,0.95,ERROR,Bank account information not available,2023-03-01,3,2023,NEW,SIN_CLABE,0c7b3b18-52e0-4678-8b81-a5638037781a,2023-03-01,1,Actual
112046,112046,3fe4adba-e86e-4c75-ba7f-d8842fb24734,Yayo,879FYFKXX,2023-03-01,0.01,ERROR,Bank account information not available,2023-03-01,3,2023,NEW,SIN_CLABE,3fe4adba-e86e-4c75-ba7f-d8842fb24734,2023-03-01,1,Actual
112044,112044,0951a0f5-2352-461a-ad22-182059e0c4a8,Bolsas Lulis,KFMSNM7JM,2023-03-01,3.00,ERROR,Bank account information not available,2023-03-01,3,2023,NEW,SIN_CLABE,0951a0f5-2352-461a-ad22-182059e0c4a8,2023-03-01,1,Actual
112043,112043,a01924a0-fbbc-4045-b5cc-738a0096d484,caminito,GJ6Q8HDVC,2023-03-01,98.70,ERROR,DEVOLUCION(17): Tipo de cuenta no corresponde ...,2023-03-01,3,2023,NEW,ERROR_USUAL,a01924a0-fbbc-4045-b5cc-738a0096d484,2023-03-01,1,Actual


In [9]:
query_error_current_month.shape

(7229, 17)

In [10]:
query_error_current_month['month'].unique()

array([3], dtype=int64)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [14]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21464\3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
97745,97745,3d768990-7ad9-478f-bf94-9fb0344c1c25,Teodora,V726PRMZR,2022-12-05,"1,437.36",ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2023-02-01,2,2023,NEW,ERROR_USUAL,3d768990-7ad9-478f-bf94-9fb0344c1c25,2023-02-01,1,NaN
101190,101190,3d768990-7ad9-478f-bf94-9fb0344c1c25,Teodora,HQRPYD5T4,2022-12-24,"5,557.78",ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2023-02-01,2,2023,NEW,ERROR_USUAL,3d768990-7ad9-478f-bf94-9fb0344c1c25,2023-02-01,1,NaN
92189,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN


In [15]:
query_new_last_month.shape

(3, 17)

# INFORMACIÓN QUERIES - PAGOS

In [16]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [17]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [18]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [19]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [20]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2023\03. MARCH\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2023\03. MARZO\QUERIES\*.csv', recursive=False)

In [21]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [22]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [23]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2023\02. FEBRUARY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2023\02. FEBRERO\QUERIES\*.csv', recursive=False)

In [24]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [25]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

### SPECIAL MONTH

In [26]:
# TRADICIONAL
trad_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2023\01. JANUARY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2023\01. ENERO\QUERIES\*.csv', recursive=False)

In [27]:
disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_spec])
disp_files_csv_2021_spec['date_query'] = disp_files_csv_2021_spec["file_name"].str[-14:-4]
disp_files_csv_2021_spec['date_query']= pd.to_datetime(disp_files_csv_2021_spec['date_query'])

In [28]:
fp_disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_spec])
fp_disp_files_csv_2021_spec['date_query'] = fp_disp_files_csv_2021_spec["file_name"].str[-14:-4]
fp_disp_files_csv_2021_spec['date_query']= pd.to_datetime(fp_disp_files_csv_2021_spec['date_query']) 

## ALL FILES

In [29]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last,
                     disp_files_csv_2021_spec,fp_disp_files_csv_2021_spec])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('SANT'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion
2924,DAILY,08fcd844-82e4-4205-a468-7e683336c565,4fa778c0-80ee-4fe4-acb8-c400c458a3e7,Los Gordos,marilu.buenrostro.gut@gmail.com,222M6LXKL,2023-01-01,293.22,"BanCoppel, S.A.",2023-01-01 12:56:47,Alison Fernanda Espinosa Aguilar,'137180104588639501,2023-01-01 10:03:35,BANORTE,"Alison Fernanda Espinosa Aguilar,,293.22,13718...",BNTE-1 2023-01-01.csv,2023-01-01,TRAD
13650,DAILY,2b4a6bf2-b9b0-4426-87e0-fc48b53f4ecc,c5c4d3c2-f05e-4cdb-8cda-7d6675295473,Abarrotes Vale,maripatlan92@gmail.com,2234MRNBM,2023-01-01,73.79,BanCoppel,2023-01-01 12:40:23,Maria Cruz Patlan Martinez,'137222104652134489,2023-01-01 10:05:09,BANORTE,"Maria Cruz Patlan Martinez,,73.79,137222104652...",BNTE-1 2023-01-01.csv,2023-01-01,TRAD
62105,DAILY,c46cc0ee-afb5-439e-8d0d-efd36d42f174,4c00548d-d7e2-48c0-bd46-dd5af46c7551,La Chuleta De Oro,joelquiroz.78@gmail.com,22394TPP2,2023-01-01,661.19,BANAMEX,2023-01-01 12:43:08,Karla Paola Cruz Saldana,'002674700839096074,2023-01-01 10:11:42,BANORTE,"Karla Paola Cruz Saldana,,661.19,0026747008390...",BNTE-1 2023-01-01.csv,2023-01-01,TRAD
47903,DAILY,97f1aecf-f34f-4f47-9030-fae5b2b3f062,1a94946c-c166-49e8-8aff-5bd97ba23258,Papas y frituras Aylin,valegaca96@gmail.com,223M4PHFP,2023-01-01,177.27,"BBVA Bancomer, S.A.",2023-01-01 12:54:59,Gerardo Garces Leon,'012180015692247530,2023-01-01 10:09:45,BANORTE,"Gerardo Garces Leon,,177.27,012180015692247530...",BNTE-1 2023-01-01.csv,2023-01-01,TRAD
22581,DAILY,478bd8e4-65d0-471e-9150-43ea9e0d14a5,cba31c03-d72e-4e48-ab7c-c030220e9892,Bienestar y Nueva Imagen,conbieni@outlook.com,223TZS3XD,2023-01-01,"1,245.71",BANORTE,2023-01-01 12:38:08,Luis Anuar Albarran Orfaly,'072441012021782908,2023-01-01 10:06:23,BANORTE,"Luis Anuar Albarran Orfaly,,1245.71,0724410120...",BNTE-1 2023-01-01.csv,2023-01-01,TRAD


In [30]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [31]:
allfiles.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion,tipo_pago
2924,DAILY,08fcd844-82e4-4205-a468-7e683336c565,4fa778c0-80ee-4fe4-acb8-c400c458a3e7,Los Gordos,marilu.buenrostro.gut@gmail.com,222M6LXKL,2023-01-01,293.22,"BanCoppel, S.A.",2023-01-01 12:56:47,Alison Fernanda Espinosa Aguilar,'137180104588639501,2023-01-01 10:03:35,BANORTE,"Alison Fernanda Espinosa Aguilar,,293.22,13718...",BNTE-1 2023-01-01.csv,2023-01-01,TRAD,Pago del día
13650,DAILY,2b4a6bf2-b9b0-4426-87e0-fc48b53f4ecc,c5c4d3c2-f05e-4cdb-8cda-7d6675295473,Abarrotes Vale,maripatlan92@gmail.com,2234MRNBM,2023-01-01,73.79,BanCoppel,2023-01-01 12:40:23,Maria Cruz Patlan Martinez,'137222104652134489,2023-01-01 10:05:09,BANORTE,"Maria Cruz Patlan Martinez,,73.79,137222104652...",BNTE-1 2023-01-01.csv,2023-01-01,TRAD,Pago del día
62105,DAILY,c46cc0ee-afb5-439e-8d0d-efd36d42f174,4c00548d-d7e2-48c0-bd46-dd5af46c7551,La Chuleta De Oro,joelquiroz.78@gmail.com,22394TPP2,2023-01-01,661.19,BANAMEX,2023-01-01 12:43:08,Karla Paola Cruz Saldana,'002674700839096074,2023-01-01 10:11:42,BANORTE,"Karla Paola Cruz Saldana,,661.19,0026747008390...",BNTE-1 2023-01-01.csv,2023-01-01,TRAD,Pago del día
47903,DAILY,97f1aecf-f34f-4f47-9030-fae5b2b3f062,1a94946c-c166-49e8-8aff-5bd97ba23258,Papas y frituras Aylin,valegaca96@gmail.com,223M4PHFP,2023-01-01,177.27,"BBVA Bancomer, S.A.",2023-01-01 12:54:59,Gerardo Garces Leon,'012180015692247530,2023-01-01 10:09:45,BANORTE,"Gerardo Garces Leon,,177.27,012180015692247530...",BNTE-1 2023-01-01.csv,2023-01-01,TRAD,Pago del día
22581,DAILY,478bd8e4-65d0-471e-9150-43ea9e0d14a5,cba31c03-d72e-4e48-ab7c-c030220e9892,Bienestar y Nueva Imagen,conbieni@outlook.com,223TZS3XD,2023-01-01,"1,245.71",BANORTE,2023-01-01 12:38:08,Luis Anuar Albarran Orfaly,'072441012021782908,2023-01-01 10:06:23,BANORTE,"Luis Anuar Albarran Orfaly,,1245.71,0724410120...",BNTE-1 2023-01-01.csv,2023-01-01,TRAD,Pago del día


In [32]:
allfiles.shape

(8862684, 19)

# CRUCE 1 - MES ACTUAL

In [33]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,08fcd844-82e4-4205-a468-7e683336c565,4fa778c0-80ee-4fe4-acb8-c400c458a3e7,Los Gordos,marilu.buenrostro.gut@gmail.com,222M6LXKL,2023-01-01,293.22,"BanCoppel, S.A.",2023-01-01 12:56:47,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,2b4a6bf2-b9b0-4426-87e0-fc48b53f4ecc,c5c4d3c2-f05e-4cdb-8cda-7d6675295473,Abarrotes Vale,maripatlan92@gmail.com,2234MRNBM,2023-01-01,73.79,BanCoppel,2023-01-01 12:40:23,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,c46cc0ee-afb5-439e-8d0d-efd36d42f174,4c00548d-d7e2-48c0-bd46-dd5af46c7551,La Chuleta De Oro,joelquiroz.78@gmail.com,22394TPP2,2023-01-01,661.19,BANAMEX,2023-01-01 12:43:08,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,DAILY,97f1aecf-f34f-4f47-9030-fae5b2b3f062,1a94946c-c166-49e8-8aff-5bd97ba23258,Papas y frituras Aylin,valegaca96@gmail.com,223M4PHFP,2023-01-01,177.27,"BBVA Bancomer, S.A.",2023-01-01 12:54:59,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,478bd8e4-65d0-471e-9150-43ea9e0d14a5,cba31c03-d72e-4e48-ab7c-c030220e9892,Bienestar y Nueva Imagen,conbieni@outlook.com,223TZS3XD,2023-01-01,"1,245.71",BANORTE,2023-01-01 12:38:08,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [34]:
cruce_1.shape

(8862684, 27)

In [35]:
duplicates_cruce_1_previo = cruce_1[cruce_1.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_previo.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
487,DAILY,c777f5ee-24ed-4f51-a70b-ca2d148d7f67,38ff8ba4-efb4-474a-9ad4-2adefce7f56d,Il Patio,memo_al@live.com.mx,27XMGM3QH,2023-01-01,"4,474.09",HSBC México,2023-01-01 12:59:46,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2159,DAILY,dc547d79-14b6-484f-9609-4426530da61b,e88a91ba-16b2-49b7-8d13-c965efaa136d,Almendros,enriquezuac78@gmail.com,2W82Z5T7K,2022-12-31,"1,246.68","Banco Santander, S.A.",2023-01-01 13:00:42,...,TRAD,Reprogramación,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2316,DAILY,f751c113-8a48-4fac-bb76-f61c2f22117a,7c272c43-94a0-4b12-9c38-6a432bfa6f07,The Stars Of Fire,lilicelis310187@gmail.com,2XY22W4ZD,2023-01-01,"1,519.76",AZTECA,2023-01-01 12:48:18,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4675,DAILY,45f839d4-348b-4e94-acae-b5841d8c9456,7519c66c-15ab-439e-87d0-27fc7dd4f55e,LOS 3 GARCIA,oscargarciavaz@gmail.com,3VF649TSV,2023-01-01,"72,835.81","Banco Mercantil del Norte, S.A.",2023-01-01 12:38:37,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
10379,DAILY,179c0fbe-4dce-4d79-8aff-6effe1e1f226,f5f407d4-a847-40b8-b726-8785a0df0c0e,El super ahorro,ruthdominguezcuevas@hotmail.com,64YBB6YNN,2023-01-01,"1,731.15","BBVA Bancomer, S.A.",2023-01-01 12:57:01,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [36]:
duplicates_cruce_1_previo.shape

(33424, 27)

In [37]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21464\1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
5171011,BLANK,1cd69e08-27c6-4014-aa8a-5e527266f71d,BLANK,BLANK,BLANK,ALG2MZ3RX,2023-02-24,"5,713.91","BBVA Bancomer, S.A.",2023-02-24 22:07:48,...,FP,Pago del día,"5,713.91",2023-03-27,2023-03-27,3.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
5173608,BLANK,f3ac2083-0b7c-4f39-8e9a-846e49c14f56,BLANK,BLANK,BLANK,AS7NFT65Q,2023-02-24,287.47,"Opciones Empresariales del Noreste, S.A. de C....",2023-02-24 22:07:23,...,FP,Pago del día,287.47,2023-03-01,2023-03-01,3.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
5492767,INSTANT,a29eeae6-ec37-4efa-a460-fca1fd681513,02211db0-689c-4d85-a804-3310b29ddd0f,Novedades y Dulces Lety,selenealarcon16@gmail.com,B7ZKDJ752,2023-02-27,114.99,"Opciones Empresariales del Noreste, S.A. de C....",2023-02-27 14:12:29,...,FP,Pago del día,114.99,2023-03-09,2023-03-09,3.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
5543564,DAILY,7d2c67ce-5d78-42e3-b665-3382792508c5,a075de10-7bf0-44ab-aea8-6f1df1149516,Carniceria Garcia,higinio.garcia.1101@gmail.com,2FRG9CR5R,2023-02-28,"1,383.68",BBVA BANCOMER,2023-03-01 05:49:50,...,TRAD,Pago del día,"1,383.68",2023-03-01,2023-03-01,3.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,ERROR_USUAL,NEW
5546305,DAILY,18802b38-caa2-47aa-952a-af1e4942a21c,eb3bbadb-363a-4f10-b604-4ffb2d8ce828,La estrella de dimas,estrelladedimas2@hotmail.com,3CV76SQMB,2023-02-28,2.85,HSBC México,2023-03-01 05:49:49,...,TRAD,Pago del día,2.85,2023-03-01,2023-03-01,3.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW


In [38]:
cruce_1_final.shape

(2038, 27)

In [39]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [40]:
duplicates_cruce_1_final.shape

(0, 27)

# CRUCE 2 MES ACTUAL

In [41]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,112083,2887ea44-8d27-45f3-9654-dadf290e75be,Freddo Condesa,D3H2CYW8R,2023-03-01,"1,176.71",ERROR,DEVOLUCION(02): Cuenta bloqueada 021180...,2023-03-01,3,2023,NEW,ERROR_USUAL,2887ea44-8d27-45f3-9654-dadf290e75be,2023-03-01,1,Actual,TRAD,Pago del día
1,112047,0c7b3b18-52e0-4678-8b81-a5638037781a,Recargas y zapato s,MMLMYGB2D,2023-03-01,0.95,ERROR,Bank account information not available,2023-03-01,3,2023,NEW,SIN_CLABE,0c7b3b18-52e0-4678-8b81-a5638037781a,2023-03-01,1,Actual,Not found,NaN
2,112046,3fe4adba-e86e-4c75-ba7f-d8842fb24734,Yayo,879FYFKXX,2023-03-01,0.01,ERROR,Bank account information not available,2023-03-01,3,2023,NEW,SIN_CLABE,3fe4adba-e86e-4c75-ba7f-d8842fb24734,2023-03-01,1,Actual,Not found,NaN
3,112044,0951a0f5-2352-461a-ad22-182059e0c4a8,Bolsas Lulis,KFMSNM7JM,2023-03-01,3.00,ERROR,Bank account information not available,2023-03-01,3,2023,NEW,SIN_CLABE,0951a0f5-2352-461a-ad22-182059e0c4a8,2023-03-01,1,Actual,Not found,NaN
4,112043,a01924a0-fbbc-4045-b5cc-738a0096d484,caminito,GJ6Q8HDVC,2023-03-01,98.70,ERROR,DEVOLUCION(17): Tipo de cuenta no corresponde ...,2023-03-01,3,2023,NEW,ERROR_USUAL,a01924a0-fbbc-4045-b5cc-738a0096d484,2023-03-01,1,Actual,TRAD,Pago del día


In [42]:
cruce_2.shape

(7885, 19)

In [43]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,112083,2887ea44-8d27-45f3-9654-dadf290e75be,Freddo Condesa,D3H2CYW8R,2023-03-01,"1,176.71",ERROR,DEVOLUCION(02): Cuenta bloqueada 021180...,2023-03-01,3,2023,NEW,ERROR_USUAL,2887ea44-8d27-45f3-9654-dadf290e75be,2023-03-01,1,Actual,TRAD,Pago del día
4,112043,a01924a0-fbbc-4045-b5cc-738a0096d484,caminito,GJ6Q8HDVC,2023-03-01,98.70,ERROR,DEVOLUCION(17): Tipo de cuenta no corresponde ...,2023-03-01,3,2023,NEW,ERROR_USUAL,a01924a0-fbbc-4045-b5cc-738a0096d484,2023-03-01,1,Actual,TRAD,Pago del día
6,112041,cd607558-3acd-4768-9714-98375947e0be,PRISKA REGALOS y PAPELERIA,P2Y54QBC7,2023-03-01,6.00,ERROR,DEVOLUCION(02): Cuenta bloqueada 012180...,2023-03-01,3,2023,NEW,ERROR_USUAL,cd607558-3acd-4768-9714-98375947e0be,2023-03-01,1,Actual,TRAD,Pago del día
10,112037,06d69e85-87fb-4c00-be3f-6286cdf6b268,Carnicera Torres,3ZGBCTRQL,2023-03-01,0.80,ERROR,DEVOLUCION(01): Cuenta inexistente 1370...,2023-03-01,3,2023,NEW,ERROR_USUAL,06d69e85-87fb-4c00-be3f-6286cdf6b268,2023-03-01,1,Actual,TRAD,Pago del día
11,112036,8f32df30-c61d-4946-820a-407e93915af0,MABO,JCSSHBXCR,2023-03-01,13.80,ERROR,DEVOLUCION(03): Cuenta cancelada 072150...,2023-03-01,3,2023,NEW,ERROR_USUAL,8f32df30-c61d-4946-820a-407e93915af0,2023-03-01,1,Actual,TRAD,Pago del día


In [44]:
cruce_2_final.shape

(2694, 19)

In [45]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
81,111749,e8931d75-4738-4457-b188-7421f6fd75ab,Abarrotes Marlene,4GHZSB8X7,2023-02-28,318.13,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2023-03-01,3,2023,NEW,ERROR_USUAL,e8931d75-4738-4457-b188-7421f6fd75ab,2023-03-01,1,Actual,TRAD,Pago del día
82,111749,e8931d75-4738-4457-b188-7421f6fd75ab,Abarrotes Marlene,4GHZSB8X7,2023-02-28,318.13,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2023-03-01,3,2023,NEW,ERROR_USUAL,e8931d75-4738-4457-b188-7421f6fd75ab,2023-03-01,1,Actual,TRAD,Pago del día
89,111992,b1f7aec0-2711-4d15-b153-d0164084c7bc,Café Express,WDTPWBGZF,2023-02-28,"2,025.73",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2023-03-01,3,2023,NEW,ERROR_USUAL,b1f7aec0-2711-4d15-b153-d0164084c7bc,2023-03-01,1,Actual,TRAD,Pago del día
90,111992,b1f7aec0-2711-4d15-b153-d0164084c7bc,Café Express,WDTPWBGZF,2023-02-28,"2,025.73",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2023-03-01,3,2023,NEW,ERROR_USUAL,b1f7aec0-2711-4d15-b153-d0164084c7bc,2023-03-01,1,Actual,TRAD,Pago del día
91,111949,e51dc8b1-edcf-4f0a-808c-a41588423aa7,Beauty Zone TG,46QKXC6NS,2023-02-28,910.33,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2023-03-01,3,2023,NEW,ERROR_USUAL,e51dc8b1-edcf-4f0a-808c-a41588423aa7,2023-03-01,1,Actual,TRAD,Pago del día


In [46]:
duplicates_cruce_2_final.shape

(1087, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [47]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x                 \
esquema_query                           ERROR_USUAL                  
status_cubeta                                 ERROR                  
tipo_dispersion                                  FP           TRAD   
tipo_pago                            Reprogramación Reprogramación   
0                         2023-03-01           0.00           0.00   
1                         2023-03-02           0.00      48,370.96   
2                         2023-03-03           0.00           0.00   
3                         2023-03-04           0.00      64,607.39   
4                         2023-03-05           0.00           0.00   

                                                                         \
esquema_query                                                             
status_cubeta            NEW                                              
tipo_dispersion           FP                        TRAD                  
tipo_pago       Pago del día Reprogramación Pago del día Reprogramación   
0                     943.16           0.00   101,116.66           0.00   
1                   1,253.24           0.00    49,726.49           9.58   
2                       0.00           0.00    76,772.80           0.00   
3                       0.00           0.00     1,526.43      13,863.32   
4                       0.00           0.00    79,851.07      26,850.76   

                            
esquema_query          All  
status_cubeta               
tipo_dispersion             
tipo_pago                   
0               102,059.82  
1                99,360.27  
2                76,772.80  
3                79,997.14  
4               106,701.83

In [48]:
group_tipo_pago.shape

(31, 8)

In [49]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only       amount_x                              \
esquema_query                         ERROR_USUAL                               
status_cubeta                               ERROR          NEW                  
tipo_pago                          Reprogramación Pago del día Reprogramación   
0                       2023-03-01           0.00   102,059.82           0.00   
1                       2023-03-02      48,370.96    50,979.73           9.58   
2                       2023-03-03           0.00    76,772.80           0.00   
3                       2023-03-04      64,607.39     1,526.43      13,863.32   
4                       2023-03-05           0.00    79,851.07      26,850.76   

                          
esquema_query        All  
status_cubeta             
tipo_pago                 
0             102,059.82  
1              99,360.27  
2              76,772.80  
3              79,997.14  
4             106,701.83

In [50]:
group_esquema.shape

(31, 5)

# CRUCE 3 - MES ANTERIOR

In [51]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,08fcd844-82e4-4205-a468-7e683336c565,4fa778c0-80ee-4fe4-acb8-c400c458a3e7,Los Gordos,marilu.buenrostro.gut@gmail.com,222M6LXKL,2023-01-01,293.22,"BanCoppel, S.A.",2023-01-01 12:56:47,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,2b4a6bf2-b9b0-4426-87e0-fc48b53f4ecc,c5c4d3c2-f05e-4cdb-8cda-7d6675295473,Abarrotes Vale,maripatlan92@gmail.com,2234MRNBM,2023-01-01,73.79,BanCoppel,2023-01-01 12:40:23,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,c46cc0ee-afb5-439e-8d0d-efd36d42f174,4c00548d-d7e2-48c0-bd46-dd5af46c7551,La Chuleta De Oro,joelquiroz.78@gmail.com,22394TPP2,2023-01-01,661.19,BANAMEX,2023-01-01 12:43:08,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,DAILY,97f1aecf-f34f-4f47-9030-fae5b2b3f062,1a94946c-c166-49e8-8aff-5bd97ba23258,Papas y frituras Aylin,valegaca96@gmail.com,223M4PHFP,2023-01-01,177.27,"BBVA Bancomer, S.A.",2023-01-01 12:54:59,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,478bd8e4-65d0-471e-9150-43ea9e0d14a5,cba31c03-d72e-4e48-ab7c-c030220e9892,Bienestar y Nueva Imagen,conbieni@outlook.com,223TZS3XD,2023-01-01,"1,245.71",BANORTE,2023-01-01 12:38:08,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [52]:
cruce_3.shape

(8862684, 27)

In [53]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21464\2713612933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2920122,DAILY,3d768990-7ad9-478f-bf94-9fb0344c1c25,51866b59-e0a8-413f-8666-991c1dafb19f,Teodora,teodoravillarojas@outlook.com,HQRPYD5T4,2022-12-24,"5,557.78","BBVA Bancomer, S.A.",2023-02-01 11:43:06,...,TRAD,Reprogramación,"5,557.78",2023-02-01,2023-02-01,2.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,ERROR_USUAL,NEW
2951385,DAILY,3d768990-7ad9-478f-bf94-9fb0344c1c25,51866b59-e0a8-413f-8666-991c1dafb19f,Teodora,teodoravillarojas@outlook.com,V726PRMZR,2022-12-05,"1,437.36","BBVA Bancomer, S.A.",2023-02-01 11:43:06,...,TRAD,Reprogramación,"1,437.36",2023-02-01,2023-02-01,2.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,ERROR_USUAL,NEW
5560374,DAILY,5e4d322e-a1d9-454a-9d2a-bef911300ca2,698079bc-9462-4dbd-bee5-a25b2b903cff,MESHICO Viajes y Eventos,meshico_direccion@hotmail.com,84VFGYN8H,2022-11-01,"3,602.98","BBVA Bancomer, S.A.",2023-02-28 11:59:28,...,TRAD,Reprogramación,"3,602.98",2023-02-28,2023-02-28,2.00,ERROR,R8,ERROR_USUAL,NEW


In [54]:
cruce_3_final.shape

(3, 27)

In [55]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [56]:
duplicates_cruce_3_final.shape

(0, 27)

# CRUCE 4 - MES ANTERIOR

In [57]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,97745,3d768990-7ad9-478f-bf94-9fb0344c1c25,Teodora,V726PRMZR,2022-12-05,"1,437.36",ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2023-02-01,2,2023,NEW,ERROR_USUAL,3d768990-7ad9-478f-bf94-9fb0344c1c25,2023-02-01,1,NaN,TRAD,Reprogramación
1,101190,3d768990-7ad9-478f-bf94-9fb0344c1c25,Teodora,HQRPYD5T4,2022-12-24,"5,557.78",ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2023-02-01,2,2023,NEW,ERROR_USUAL,3d768990-7ad9-478f-bf94-9fb0344c1c25,2023-02-01,1,NaN,TRAD,Reprogramación
2,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación
3,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación
4,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación


In [58]:
cruce_4.shape

(10, 19)

In [59]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,97745,3d768990-7ad9-478f-bf94-9fb0344c1c25,Teodora,V726PRMZR,2022-12-05,"1,437.36",ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2023-02-01,2,2023,NEW,ERROR_USUAL,3d768990-7ad9-478f-bf94-9fb0344c1c25,2023-02-01,1,NaN,TRAD,Reprogramación
1,101190,3d768990-7ad9-478f-bf94-9fb0344c1c25,Teodora,HQRPYD5T4,2022-12-24,"5,557.78",ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2023-02-01,2,2023,NEW,ERROR_USUAL,3d768990-7ad9-478f-bf94-9fb0344c1c25,2023-02-01,1,NaN,TRAD,Reprogramación
2,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación
3,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación
4,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación


In [60]:
cruce_4_final.shape

(10, 19)

In [61]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
2,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación
3,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación
4,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación
5,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación
6,92189,5e4d322e-a1d9-454a-9d2a-bef911300ca2,MESHICO Viajes y Eventos,84VFGYN8H,2022-11-01,"3,602.98",ERROR,R8,2023-02-28,2,2023,NEW,ERROR_USUAL,5e4d322e-a1d9-454a-9d2a-bef911300ca2,2023-02-28,28,NaN,TRAD,Reprogramación


In [62]:
duplicates_cruce_4_final.shape

(8, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [63]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ultimo_mov_date_only       amount_x          
esquema_query                           ERROR_USUAL       All
status_cubeta                                   NEW          
tipo_dispersion                                TRAD          
tipo_pago                            Reprogramación          
0                         2023-02-01       6,995.14  6,995.14
1                         2023-02-28       3,602.98  3,602.98
2                                All      10,598.12 10,598.12

In [64]:
group_tipo_pago_last_month.shape

(3, 3)

In [65]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ultimo_mov_date_only         amount          
esquema_query                         ERROR_USUAL       All
status_cubeta                                 NEW          
tipo_pago                          Reprogramación          
0                       2023-02-01       6,995.14  6,995.14
1                       2023-02-28      28,823.84 28,823.84
2                              All      35,818.98 35,818.98

In [66]:
group_esquema_last_month.shape

(3, 3)

In [67]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL MARZO 2023.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()